In [1]:
import cvxpy as cp
import numpy as np
import time

In [2]:
# network structure
class TrafficNet:
    __slots__ = ('zone', 'trip', 'node', 'link', 'path', 'log', 'node_seq', 'link_seq', 'path_seq')

    def __init__(self):
        """
        Traffic network components.
        """
        self.zone = {}
        self.trip = {}
        self.node = {}
        self.link = {}
        self.path = {}
        self.log = {}  # Log for recording all the algorithmic information.
        self.node_seq = 0
        self.link_seq = 0
        self.path_seq = 0

    class Zone:

        __slots__ = ('Id', 'desList')

        def __init__(self, Id):
            self.Id = Id - 1
            self.desList = []

    class Trip:

        __slots__ = ('ori', 'des', 'dem', 'cost', 'paths', 'BB_step_size')

        def __init__(self, ori, des, dem=0.0):
            self.ori = ori
            self.des = des
            self.dem = dem
            self.cost = 0.0
            self.paths = []
            """Algorithm"""
            self.BB_step_size = 1

    class Node:
        """
        This class has attributes associated with any node
        """
        __slots__ = ('Id', 'lat', 'lon', 'outLinks', 'inLinks')

        def __init__(self, Id, lat=0.0, lon=0.0):
            self.Id = Id - 1
            self.lat = lat
            self.lon = lon
            self.outLinks = []
            self.inLinks = []

    class Link:
        """
        This class has attributes associated with any link
        """
        __slots__ = ('Id', 'tail', 'head', 'cap', 'length', 'fft', 'alpha', 'beta', 'speedLimit', 'toll', 'type',
                     'flow', 'time', 'cost', 'deri', 'integral', 'last_flow', 'Armijo_flow', 'Armijo_integral')

        def __init__(self, tail, head, cap=0.0, length=0.0, fft=0.0, alpha=0.15, beta=4, speedLimit=0.0, toll=0.0, type=1):
            """Input data"""
            self.Id = 0
            self.tail = tail
            self.head = head
            self.cap = cap
            self.length = length
            self.fft = fft
            self.alpha = alpha  # BPR function
            self.beta = beta  # BPR function
            self.speedLimit = speedLimit
            self.toll = toll
            self.type = type
            """Algorithm"""
            self.flow = 0.0
            self.time = fft  # initial setting
            self.cost = fft  # initial setting
            self.deri = 0.0
            self.integral = 0.0
            self.last_flow = 0.0
            self.Armijo_flow = 0.0
            self.Armijo_integral = 0.0

    def add_trip(self, ori, des, dem):
        """
        Add trips and zones.
        """
        if dem > 0:
            self.trip[ori, des] = self.Trip(ori, des, dem)
            if ori not in self.zone:
                self.zone[ori] = self.Zone(ori)
            if des not in self.zone:
                self.zone[des] = self.Zone(des)
            if des not in self.zone[ori].desList:
                self.zone[ori].desList.append(des)

    def add_node(self, Id, lat=0.0, lon=0.0):
        """
        Add nodes.
        """
        self.node[Id] = self.Node(Id, lat, lon)

    def add_link(self, tail, head, cap=0.0, length=0.0, fft=0.0, alpha=0.15, beta=4, speedLimit=0.0, toll=0.0, type=1):
        """
        Add links and nodes.
        """
        self.link[tail, head] = self.Link(tail, head, cap, length, fft, alpha, beta, speedLimit, toll, type)
        self.link[tail, head].Id = self.link_seq
        self.link_seq += 1
        # create node and set up node-link incidence relationship.
        if tail not in self.node:
            self.node[tail] = self.Node(tail)
        if head not in self.node:
            self.node[head] = self.Node(head)
        if (tail, head) not in self.node[tail].outLinks:
            self.node[tail].outLinks.append((tail, head))
        if (tail, head) not in self.node[head].inLinks:
            self.node[head].inLinks.append((tail, head))

In [3]:
# data
netFile = 'data\\Anaheim\\network.csv'
demFile = 'data\\Anaheim\\demand.csv'

In [4]:
# read network
'''Instantiation'''
TN = TrafficNet()

'''Read network'''
with open(netFile, 'r') as inFile:
    next(inFile)  # skip the first title line
    for line in inFile:
        # data tail,head,capacity,length,fft,alpha,beta,speedLimit,toll,link_type
        tmpIn = line.strip('\n').split(',')
        tail = int(tmpIn[0])
        head = int(tmpIn[1])
        capacity = float(tmpIn[2])
        length = float(tmpIn[3])
        fft = float(tmpIn[4])
        alpha = float(tmpIn[5])
        beta = float(tmpIn[6])
        speedLimit = float(tmpIn[7])
        toll = float(tmpIn[8])
        link_type = int(tmpIn[9])
        TN.add_link(tail, head, cap=capacity, length=length, fft=fft, alpha=alpha, beta=beta, speedLimit=speedLimit, toll=toll, type=link_type)

'''Read demand'''
with open(demFile, 'r') as inFile:
    next(inFile)  # skip the first title line
    for line in inFile:
        # data
        tmpIn = line.strip('\n').split(',')
        ori = int(tmpIn[0])
        des = int(tmpIn[1])
        demand = float(tmpIn[2])
        TN.add_trip(ori, des, demand)

In [5]:
# solve the node-link formulation of UE
x = cp.Variable(len(TN.link), nonneg=True)
xr = cp.Variable((len(TN.zone), len(TN.link)), nonneg=True)

tmp = 0
for l in TN.link:
    tmp += TN.link[l].fft * x[TN.link[l].Id] + TN.link[l].fft * TN.link[l].alpha * TN.link[l].cap / (
        TN.link[l].beta + 1) * (x[TN.link[l].Id] / TN.link[l].cap) ** (TN.link[l].beta + 1)
obj_func = cp.Minimize(tmp)

constraints = []
for r in TN.zone:
    for j in TN.node:
        if j == r:
            qrj = -sum(TN.trip[r, s].dem for s in TN.zone[r].desList)
        elif j in TN.zone[r].desList:
            qrj = TN.trip[r, j].dem
        else:
            qrj = 0
        constraints += [sum(xr[TN.zone[r].Id, TN.link[l].Id] for l in TN.node[j].inLinks) 
                        - sum(xr[TN.zone[r].Id, TN.link[l].Id] for l in TN.node[j].outLinks) == qrj]

for l in TN.link:
    constraints += [x[TN.link[l].Id] == sum(xr[TN.zone[r].Id, TN.link[l].Id] for r in TN.zone)]

problem = cp.Problem(obj_func, constraints)

start = time.time()
problem.solve(verbose=True, abstol=1e-4)
end = time.time()
print('Solving time:', end - start)

d:\Software\Anaconda\Lib\site-packages\cvxpy\problems\problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) May 21 04:50:02 PM: Your problem has 35646 variables, 16722 constraints, and 0 parameters.
(CVXPY) May 21 04:50:04 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 21 04:50:04 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 21 04:50:04 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 21 04:50:04 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 21 04:50:06 PM: Compiling problem (target solver=ECOS

d:\Software\Anaconda\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


(CVXPY) May 21 04:50:10 PM: Applying reduction CvxAttr2Constr
(CVXPY) May 21 04:50:13 PM: Applying reduction ConeMatrixStuffing
(CVXPY) May 21 04:51:39 PM: Applying reduction ECOS
(CVXPY) May 21 04:51:46 PM: Finished problem compilation (took 1.018e+02 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) May 21 04:51:46 PM: Invoking solver ECOS  to obtain a solution.
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) May 21 04:52:53 PM: Problem status: optimal
(CVXPY) May 21 04:52:53 PM: Optimal value: 1.206e+06
(CVXPY) May 21 04:52:53 PM: Compilation took 1.018e+02 seconds
(CVXPY) May 21 04

In [6]:
for ind, l in enumerate(TN.link):
        print(f"link {l}, flow = {x.value[ind]}")

link (1, 117), flow = 7074.900017256685
link (2, 87), flow = 9662.500010476286
link (3, 74), flow = 7669.000019385322
link (4, 233), flow = 12173.800032054602
link (5, 165), flow = 2586.8000276756243
link (6, 213), flow = 6576.600022382722
link (7, 253), flow = 7137.100026900768
link (8, 411), flow = 722.1000134046062
link (9, 379), flow = 1420.087002046239
link (9, 395), flow = 817.4130256261785
link (10, 338), flow = 105.0864234949106
link (10, 362), flow = 44.21360436455654
link (11, 309), flow = 485.8000134049115
link (12, 275), flow = 488.20001339881065
link (13, 262), flow = 37.00001339872953
link (14, 257), flow = 125.20001340508185
link (15, 254), flow = 407.10001314690896
link (16, 263), flow = 249.00001339982984
link (17, 276), flow = 648.3000133956293
link (18, 322), flow = 2218.300019296169
link (18, 348), flow = 650.5000108578732
link (19, 364), flow = 723.5040303410786
link (19, 380), flow = 314.49599935915023
link (20, 397), flow = 503.6000117332405
link (21, 412), flow 

The CVXPY solver cannot get a solution for the UE problem on the Winnipeg network within 2 hours.